In [1]:
from seal import *
import numpy as np
import copy

def print_vector(vector):
    print('[ ', end='')
    for i in range(0, 8):
        print(vector[i], end=', ')
    print('... ]')

def example_bgv_basics():
    parms = EncryptionParameters (scheme_type.bgv)
    poly_modulus_degree = 8192
    parms.set_poly_modulus_degree(poly_modulus_degree)
    parms.set_coeff_modulus(CoeffModulus.BFVDefault(poly_modulus_degree))
    parms.set_plain_modulus(PlainModulus.Batching(poly_modulus_degree, 20))
    context = SEALContext(parms)

    keygen = KeyGenerator(context)
    secret_key = keygen.secret_key()
    public_key = keygen.create_public_key()
    relin_keys = keygen.create_relin_keys()
    galois_keys = keygen.create_galois_keys()      # Galois Keys anlegen

    encryptor = Encryptor(context, public_key)
    evaluator = Evaluator(context)
    decryptor = Decryptor(context, secret_key)

    batch_encoder = BatchEncoder(context)
    slot_count = batch_encoder.slot_count()
    row_size = slot_count / 2
    print(f'Plaintext matrix row size: {row_size}')

    pod_matrix = [0] * slot_count
    pod_matrix[0] = 1.0
    pod_matrix[1] = 2.0
    pod_matrix[2] = 3.0
    
    pod_matrix2 = [0] * slot_count
    pod_matrix2[0] = 2.0
    pod_matrix2[1] = 3.0
    pod_matrix2[2] = 4.0

    p1 = batch_encoder.encode(pod_matrix)
    p2 = batch_encoder.encode(pod_matrix2)

    c1 = encryptor.encrypt(p1)
    c2 = encryptor.encrypt(p2)

    p1 = decryptor.decrypt(c1)
    r1 = batch_encoder.decode(p1)
    
    p2 = decryptor.decrypt(c2)
    r2 = batch_encoder.decode(p2)
    
    ccSum = evaluator.add(c1, c2)
    pccSum = decryptor.decrypt(ccSum)
    rccSum = batch_encoder.decode(pccSum)
    
    ccSub = evaluator.sub(c1, c2)
    pccSub = decryptor.decrypt(ccSub)
    rccSub = batch_encoder.decode(pccSub)
    
    ccMul = evaluator.multiply(c1, c2)
    pccMul = decryptor.decrypt(ccMul)
    rccMul = batch_encoder.decode(pccMul)

    cSq = evaluator.square(c1)
    pcSq = decryptor.decrypt(cSq)
    rcSq = batch_encoder.decode(pcSq)
    
    cNeg = evaluator.negate(c1)
    pcNeg = decryptor.decrypt(cNeg)
    rcNeg = batch_encoder.decode(pcNeg)
    
    # NOTE: MSSEAL DOES NOT HAVE A DESIGNATED POWER FCT
    cPow = evaluator.multiply(c1, c1)
    for i in range(3-2):
        cPow = evaluator.multiply(cPow, c1)
    pcPow = decryptor.decrypt(cPow)
    rcPow = batch_encoder.decode(pcPow)
        
    cpSum = evaluator.add_plain(c1, p2)
    pcpSum = decryptor.decrypt(cpSum)
    rcpSum = batch_encoder.decode(pcpSum)
    
    cpSub = evaluator.sub_plain(c1, p2)
    pcpSub = decryptor.decrypt(cpSub)
    rcpSub = batch_encoder.decode(pcpSub)
    
    cpMul = evaluator.multiply_plain(c1, p2)
    pcpMul = decryptor.decrypt(cpMul)
    rcpMul = batch_encoder.decode(pcpMul)
        
    cRotR = evaluator.rotate_rows(c1, 1, galois_keys)
    pcRotR = decryptor.decrypt(cRotR)
    rcRotR = batch_encoder.decode(pcRotR)
    
    print('r1:     {}'.format(r1))
    print('r2:     {}'.format(r2))
    print('rccSum: {}'.format(rccSum))
    print('rccSub:  {}'.format(rccSub))
    print('rccMul: {}'.format(rccMul))
    print('rcSq:   {}'.format(rcSq))
    print('rcNeg:  {}'.format(rcNeg))
    print('rcPow:  {}'.format(rcPow))
    print('rcpSum: {}'.format(rcpSum))
    print('rcpSub: {}'.format(rcpSub))
    print('rcpMul: {}'.format(rcpMul))
    print('rcRotR: {}'.format(rcRotR))


if __name__ == "__main__":
    example_bgv_basics()


Plaintext matrix row size: 4096.0
r1:     [1 2 3 ... 0 0 0]
r2:     [2 3 4 ... 0 0 0]
rccSum: [3 5 7 ... 0 0 0]
rccSub:  [-1 -1 -1 ...  0  0  0]
rccMul: [ 2  6 12 ...  0  0  0]
rcSq:   [1 4 9 ... 0 0 0]
rcNeg:  [-1 -2 -3 ...  0  0  0]
rcPow:  [ 1  8 27 ...  0  0  0]
rcpSum: [3 5 7 ... 0 0 0]
rcpSub: [-1 -1 -1 ...  0  0  0]
rcpMul: [ 2  6 12 ...  0  0  0]
rcRotR: [2 3 0 ... 0 0 0]
